Load model first: requires to change "model_path" variable

In [2]:
import torch
import numpy as np
from ultralytics import YOLO
import os

model_path = "./best.pt"
model = YOLO(model_path)

In [ ]:
import pandas as pd

test_folder = "./thorax_dataset/images/test/"
csv_file_name = "thorax_only.csv"
df = pd.read_csv(csv_file_name)
df.head()

In [ ]:
# List all image file names inside the folder
image_files = set(os.listdir(test_folder))

# Filter the DataFrame to retain only rows with filenames in the folder
filtered_df = df[df['ant'].isin(image_files)]
filtered_df.head()

Predict the landmarks locations present inside the test dataset

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

ground_truth = filtered_df.iloc[:, 1:].to_numpy()
# Nx4 Matrix to store the landmarks coordinates (each rows are [x_head, y_head, x_abdomen, y_abdomen])
prediction_data = np.zeros((ground_truth.shape[0], 4))

# Vector that contains 1 and 0 for ants images with detected and not detected landkmarks respectivelly 
is_detected_data = np.ones(ground_truth.shape[0])

for i, img in enumerate(filtered_df.iloc[:, 0]):
    path = test_folder + img

    results = model.predict(path)
    shape = results[0].keypoints.shape
    if shape[1] == 0:
        is_detected_data[i] = 0
    else:
        point1_pred = results[0].keypoints.xy[0][0].cpu().numpy()
        point2_pred = results[0].keypoints.xy[0][1].cpu().numpy()
        
        prediction_data[i, 0:2] = point1_pred
        prediction_data[i, 2:] = point2_pred

In this cell below we compare the predictions of our finetuned model and the ground truth.

In [ ]:
def show_image_and_landmarks(index, path, prediction_data, ground_truth):    
    img = mpimg.imread(path)

    plt.imshow(img)
    plt.axis('on')  # Turn on axes for reference

    point1_pred = prediction_data[index, 0:2]
    point2_pred = prediction_data[index, 2:]

    point1_ground_thruth = ground_truth[index, 0:2]
    point2_ground_thruth = ground_truth[index, 2:]

    # Plot the points

    plt.scatter(*point1_pred, color='red', marker='D',label=f'Point 1 pred: {point1_pred}')
    plt.scatter(*point2_pred, color='red', label=f'Point 2 pred: {point2_pred}')

    plt.scatter(*point1_ground_thruth, color='blue', marker='D',label=f'Point 1 truth: {point1_ground_thruth}')
    plt.scatter(*point2_ground_thruth, color='blue', label=f'Point 2 truth: {point2_ground_thruth}')

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()

def compute_WL(data):
    x1 = data[:,0]
    y1 = data[:,1]

    x2 = data[:,2]
    y2 = data[:,3]

    return ((x1-x2)**2 + (y1-y2)**2)**0.5

pred  = prediction_data[is_detected_data == 1, :]
GT  = ground_truth[is_detected_data == 1, :]

#Matrix that contains the difference (in pixel) between the predicted and ground truth landmark
difference_landmarks = pred-GT

i = 1
files = filtered_df.iloc[:, 0]
show_image_and_landmarks(i, test_folder+str(files.iloc[i]), prediction_data, ground_truth)


print("example landmark ant " + str(files.iloc[i]))
print("prediction:")
print(prediction_data[i,:])
print("ground truth:")
print(GT[i,:])
print("difference:")
d = prediction_data-ground_truth
print(d[i,:])


print("------------------------------------")
# Webber length
WL_pred =  compute_WL(pred)
WL_GT =  compute_WL(GT)

percentage_error = abs(WL_GT-WL_pred)/WL_GT
print("percentage of error Webber length:")
print(np.mean(percentage_error))

head_dist_pred_GT = ((difference_landmarks[:,0])**2 + (difference_landmarks[:,1])**2)**0.5
abdomen_dist_pred_GT = ((difference_landmarks[:,3])**2 + (difference_landmarks[:,0])**2)**0.5
print("")
print("Average distance between predicted and true head landmark:")
print(np.mean(head_dist_pred_GT))
print("Average distance (normalized by WL) between predicted and true head landmark:")
print(np.mean(abdomen_dist_pred_GT/WL_GT))
print("")

print("Average distance between predicted and true abdomen landmark:")
print(np.mean(abdomen_dist_pred_GT))
print("Average distance (normalized by WL) between predicted and true abdomen landmark:")
print(np.mean(head_dist_pred_GT/WL_GT))
print("")

print("average signed bias (pred-GT) of coordinates:")
print("(x_head, y_head, x_abdomen, y_abdomean)")
print(np.mean(difference_landmarks, axis=0))



print("average absolute bias of coordinates (x_head, y_head, x_abdomen, y_abdomean)")
print(np.mean(abs(difference_landmarks), axis=0))

Plot results images

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

max_iterations = 10

for i, file_name in enumerate(files):
    path = test_folder + file_name

    results = model.predict(path)
    shape = results[0].keypoints.shape
    
    img = mpimg.imread(path)

    plt.imshow(img)
    plt.axis('on')  # Turn on axes for reference

    if is_detected_data[i] == 1: 
        point1_pred = prediction_data[i, 0:2]
        point2_pred = prediction_data[i, 2:]

        point1_ground_thruth = ground_truth[i, 0:2]
        point2_ground_thruth = ground_truth[i, 2:]

        # Plot the points

        plt.scatter(*point1_pred, color='red', marker='D',label=f'Point 1 pred: {point1_pred}')
        plt.scatter(*point2_pred, color='red', label=f'Point 2 pred: {point2_pred}')

        plt.scatter(*point1_ground_thruth, color='blue', marker='D',label=f'Point 1 truth: {point1_ground_thruth}')
        plt.scatter(*point2_ground_thruth, color='blue', label=f'Point 2 truth: {point2_ground_thruth}')

        # Add legend
        plt.legend()

        # Show the plot
    plt.show()
    if (max_iterations == i + 1):
        break


# Heatmap generation

In [1]:
from utils import produce_heatmap

produce_heatmap("best.pt", "./thorax_dataset/images/test/casent0217282_p_1.jpg", "casent0217282_p_1.jpg", view_img=True)

Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': 2, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'imw': 640, 'imh': 640, 'model': 'best.pt', 'view_img': True}

0: 640x640 1 thorax, 12.6ms
Speed: 3.0ms preprocess, 12.6ms inference, 306.1ms postprocess per image at shape (1, 3, 640, 640)
Heatmap saved at: c:\Users\elyes\Documents\ml-project-2-win-function-p2\heatmaps\casent0217282_p_1.jpg
